In [3]:
import autograd
import autograd.numpy as np
import matplotlib.pyplot as plt
from PIL import Image 
import os
import IPython.display
import time
from astropy.nddata.utils import Cutout2D
from scipy import ndimage
import pickle

In [8]:
#extraction stuff
#Basic data structure for holding bubble information
class BubbleEvent:
    def __init__(self, File):
        #temp pixel arrays and event level meta data
        self.FileName = File
        Bot1PixelArray, Bot2PixelArray = BubbleEvent.GetPixelArray(self.FileName) #gets 2d array of pixel intensities
        self.Date, self.Run, self.EventID = BubbleEvent.GetRunInfo(self.FileName) #parses image name to get event info
        self.BubbleCount = 0
        #actual features to use to classify
        self.UsefulEdgeFeature = np.sum(GetEdgeFeature(DownSampleTheArray(2, Bot1PixelArray)) + 
                                        GetEdgeFeature(DownSampleTheArray(2, Bot2PixelArray))) #edge detect. sum
        self.UsefulBlobFeature = GetBlobs(Bot1PixelArray) + GetBlobs(Bot2PixelArray) #blob convalution deviation
        self.CountBlobPeakFeature = GetPeaks(Bot1PixelArray) + GetPeaks(Bot2PixelArray)
    def GetPixelArray(FileName):
        im = Image.open(FileName)
        PixelArray = np.asarray(im)
        Cutout = Cutout2D(PixelArray, (530,140), 235) #just cut out the parts of the image with bottles
        Bot1PixelArray = Cutout.data
        PixelArray =ndimage.rotate(PixelArray, -45)
        Cutout2 = Cutout2D(PixelArray, (270,310), 235) #other bottle view
        Bot2PixelArray = Cutout2.data
        return Bot1PixelArray, Bot2PixelArray
    def GetRunInfo(File):
        Date = int(File.split("/")[-1].split("_")[0]) #file should be date_run_event
        Run = int(File.split("/")[-1].split("_")[1])
        Event = int("{}{}{}".format(Date, Run,File.split("/")[-1].split("_")[2])) 
        return Date, Run, Event
#Functions that extract useful features from a pixel array
def Convolve(PixelArray, Kernel): #Convolve a given kernel with an array
    Size = int((np.shape(Kernel)[0]-1)/2)
    i = Size
    j = Size
    #2*size since we cut out at two ends
    Convalution = np.zeros((np.shape(PixelArray)[0]-2*Size, np.shape(PixelArray)[1]-2*Size))
    FillI=0
    FillJ=0
    while(i < np.shape(PixelArray)[0]-Size):
        j = Size
        FillJ=0
        while(j < np.shape(PixelArray)[1]-Size):
            ConvalutionResult = PixelArray[i-Size:i+Size+1, j-Size:j+Size+1]*Kernel
            Convalution[FillI,FillJ] = ConvalutionResult.sum()
            FillJ = FillJ + 1
            j = j + 1
        FillI = FillI + 1
        i = i + 1
    return Convalution
#kernel for 
def LaplaceOfGaussKernel(x, y, sigma):
    PointsX, PointsY = np.meshgrid(x,y)
    r = PointsX**2 + PointsY**2
    LoG = -1/(np.pi*sigma**4)*(1 - r/(2*sigma**2))*np.exp(-r/(2*sigma**2))
    return LoG*100

def GetBlobs(PixelArray): #run blob detection and then summarize result
    I = np.arange(-5,6)
    J = np.arange(-5,6)
    Kernel=LaplaceOfGaussKernel(I, J, 8.3)
    Convalution = Convolve(PixelArray, Kernel)
    return np.std(Convalution)

def GetPeaks(Array):
    I = np.arange(-5,6)
    J = np.arange(-5,6)
    Kernel=LaplaceOfGaussKernel(I, J, 8.3)
    Convalution = Convolve(Array, Kernel)
    Peaks = FindPeaks(Array, 15)
    return Peaks

def FindPeaks(Array, boxsize): #finds the number of elements that are bigger than all neighbors in some box
    FoundPeak = False
    PeakCount=0
    for i in range(boxsize, np.shape(Array)[0]-boxsize):
        for j in range(boxsize, np.shape(Array)[1]-boxsize):
            CurrentElem = Array[i,j]
            BoxElements = Array[i-boxsize:i+boxsize+1, j-boxsize:j+boxsize]
            if(np.max(BoxElements)<=CurrentElem and CurrentElem>np.mean(Array)+3):
                FoundPeak=True
            if(FoundPeak):
                PeakCount=PeakCount+1
                FoundPeak = False #reset the bool
    return PeakCount

def GetEdgeFeature(PixelArray): #edge detection kernel. Can be shortened by rewriting it to use convolve func
    HorizontalKernal = np.array([[1,0,-1],[2,0,-2], [1,0,-1]])
    VerticalKernal = HorizontalKernal.T
    EdgeArray = np.zeros(3) 
    Step = 3
    i=0
    j=0
    Significant = 35
    while(i < np.shape(PixelArray)[0]-2):
        j=0
        while(j < np.shape(PixelArray)[1]-2):
            XConvalution = PixelArray[i:i+Step, j:j+Step]*HorizontalKernal
            XConvalution = np.sum(XConvalution)
            YConvalution = PixelArray[i:i+Step, j:j+Step]*VerticalKernal
            YConvalution = np.sum(YConvalution)
            if(np.abs(XConvalution)>Significant and np.abs(YConvalution)<=Significant):
                EdgeArray[0] = EdgeArray[0]+ np.abs(XConvalution) #big horizontal edge
            elif(np.abs(XConvalution)<=Significant and np.abs(YConvalution)>Significant):
                EdgeArray[1] = EdgeArray[1]+ np.abs(YConvalution) #big verical edge
            elif(np.abs(XConvalution)>Significant and np.abs(YConvalution)>Significant):
                EdgeArray[2] = EdgeArray[2]+ np.abs(XConvalution) + np.abs(YConvalution) #big diagonal
            j=j+1
        i=i+1
    return EdgeArray

def DownSampleTheArray(BoxSize, Array):
    NewArray = np.zeros((int(np.shape(Array)[0]/BoxSize), int(np.shape(Array)[1]/BoxSize)))
    NewI=0
    NewJ=0
    i=0
    j=0
    while(NewI<np.shape(NewArray)[0]):
        NewJ=0
        j=0
        while(NewJ<np.shape(NewArray)[1]):
            NewArray[NewI, NewJ] = np.mean(Array[i:i+BoxSize, j:j+BoxSize])
            j = j + BoxSize
            NewJ = NewJ+1
        i = i + BoxSize
        NewI = NewI+1
    return NewArray

def GetAllDiffImages():
    path = "../bubbleimages/difffirst/"
    Files = np.array(["{}{}".format(path,file) for file in os.listdir(path) if os.path.isfile(os.path.join(path, file))])
    return Files

def GetBubbleCount(Event, BubbleInfo):
    EventID = Event.EventID
    Cut = BubbleInfo[:,0]==EventID
    if(Cut.sum()!=1): #some events have 2 different bubble count entries
        Event.BubbleCount=9999
    else:
        Event.BubbleCount = int(BubbleInfo[Cut,1])

In [ ]:
BubbleCountInfo = np.genfromtxt("../bubbleimages/bubnums.csv", delimiter=",")
DiffImages = GetAllDiffImages()
Events = []
for i in range(0, 2800):
    Event = BubbleEvent(DiffImages[i])
    GetBubbleCount(Event, BubbleCountInfo)
    if(Event.BubbleCount == 9999):
        SkippedEventCount = SkippedEventCount + 1
        continue
    Events.append(Event)
Events = np.asarray(Events, dtype=BubbleEvent)
pickel.dump(Events, open("Events.p", "w"))

In [ ]:
#use Events[i].UsefulEdgeFeature, and Events[i].UsefulBlobFeature to build classifier
#classifier stuff